## Market Impact and Trade Sizing
J. White \
5 March 2020

We want to explore the effects of market impact in an Optimal Expected Utility sizing framework

### Assumptions
* We can only invest in one risky asset S and a risk-free asset R with 0 expected return
* $\frac{\Delta S_T}{S_0}$ can take values $x_i$ with probabilities $p_i$
* Investor has CRRA utility: $$u(w) = \frac{w^{1-\gamma}}{1-\gamma}$$
* Market impact is treated as a tax, which is a function of trade size and execution duration: $$\tau = f(\kappa, \Delta t)$$
* $\tau$ is symmetric between entering and exiting
* No rebalancing between times 0 and T

### General Framework
For a wealth fraction $\kappa$ invested in $S$, we have expected utility:
$$E[u] = \frac{1}{1-\gamma}\sum_i p_i (1+\kappa x_i - 2\tau(\kappa))^{1-\gamma}$$

As usual we want to optimize wrt $\kappa$ so:
$$\frac{\partial E[u]}{\partial \kappa} = \sum_i \frac{p_i (x_i - 2\frac{\partial\tau}{\partial\kappa})}{(1+\kappa x_i - 2\tau(\kappa))^{\gamma}}  = 0 $$

This can be solved analytically for certain distributions and $\tau$ specs, but in general we'll just solve it numerically.

In [83]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Definitions
util = lambda x: (pow(x, 1 - gamma) - 1) / (1 - gamma)
def tau(k, a):
    return a * k

def expected_util(p, x, k, a, gamma):
    return sum(p * (pow((1 + k * x - 2 * tau(k, a)), 1 - gamma) - 1)) / (1 - gamma)

# Output
def plot_utils(p0=0.25, p1=0.5, p2=0.25, x0=-0.2, x1=0.1, x2=0.4, tau_strength=0.02, gamma=2):
    p = np.asarray([p0, p1, p2]) / (p0 + p1 + p2)
    x = np.asarray([x0, x1, x2])
    dk = 0.025
    N = 100
    eU_base = np.zeros(N)
    eU_impact = np.zeros(N)
    x_axis = np.zeros(N)
    for i,k in enumerate(range(0, N)):
        x_axis[i] = dk * k
        eU_base[i] = expected_util(p, x, dk * k, 0, gamma)
        eU_impact[i] = expected_util(p, x, dk * k, tau_strength, gamma)
    plt.plot(x_axis, eU_base, 'b', label='No Trade Impact')
    plt.plot(x_axis, eU_impact, 'g', label='With Trade Impact')
    plt.plot(x_axis, np.zeros(N), 'k')
    plt.xlabel('Wealth % (Trade Size)')
    plt.ylabel('Risk-adjusted Return')
    plt.legend()
    plt.show()
    print('Expected Return (pre-Impact) = {}'.format(np.dot(p,x)))
    print('Trade Impact at 100% size = {}'.format(2 * tau(1, tau_strength)))

### Interactive Utlity Graphs
* $p_i$ and $x_i$ are slider inputs and for ease are set to triplets.  Can easily be changed to 5 states to match the actual forecast procedure
* $\tau$ is simply linear in trade size with the slope an input slider - easy to put in more complicated/realistic functions in the definition of $\tau$ below
* $p_i$ sliders are unconstrained then $\bar{p}$ is normalized to 1, so $p$ sliders can be interpreted as relative probability weights

In [84]:
w=interactive(plot_utils, p0=(0,1,0.01), p1=(0,1,0.01), p2=(0,1,0.01), x0=(-1,1,0.01), x1=(-1,1,0.01), x2=(-1,1,0.01),
              tau_strength=(0,0.1,0.01), gamma=(1.01,10,0.1))
w

interactive(children=(FloatSlider(value=0.25, description='p0', max=1.0, step=0.01), FloatSlider(value=0.5, de…